In [1]:
import requests
from lxml import etree
import re
import json
import pandas as pd

In [2]:
rw_csv = pd.read_csv('rw_csv.csv')

In [3]:
falcon_url = "https://www.inoreader.com/stream/user/1004920918/tag/rw_monitor%3A%20FalconFeedsIO/view/json"

# fetch the RSS feed
response = requests.get(falcon_url)
rss_json = response.json()

In [4]:
rw_list = []
for item in rss_json['items']:
    if "#ransomware" in item['title']:
        tweet = item['title'].replace('\n',' ')
        link = item['url']
        ransomware = tweet.split('#ransomware')[0]
        date_published = item['date_published'][:10]
        date_published = pd.to_datetime(date_published).strftime('%Y-%m-%d')
        pattern = r"added (.*?) to their victim list"
        try:
            victim = re.search(pattern, tweet).group(1)
            # if "," in victim:
            #     victim = victim.split(',')[0]
            rw_list.append({'victim': victim,
            'ransomware': ransomware,
            'date_published':date_published,
            'link':link})
        except:
            next
                # print(f'Ransomware: {ransomware}\nVictim: {victim}\nLink: {link}\nDate: {date_published}')
        
# rw_df = pd.DataFrame(rw_list)
# rw_df

In [5]:
threatmon_url = 'https://www.inoreader.com/stream/user/1004920918/tag/rw_monitor%3A%20threatmon/view/json'

# fetch the RSS feed
response = requests.get(threatmon_url)
rss_json = response.json()

In [6]:
# rw_list = []
for item in rss_json['items']:
    if "Actor :" in item['title']:
        tweet = item['title'].replace('\n',' ')
        link = item['url']
        ransomware = tweet.split(' Victim :')[0].replace("Actor : ",'').strip()
        date_published = pd.to_datetime(item['date_published'][:10]).strftime('%Y-%m-%d').strip()
        try:
            victim = tweet.split(' Victim : ')[1].split(" Date")[0].strip()
            # print(f'{ransomware} | {victim} | {date_published}')
            rw_list.append({'victim': victim,
            'ransomware': ransomware,
            'date_published':date_published,
            'link':link})
        except:
            next
rw_df = pd.DataFrame(rw_list)
rw_df

,victim,ransomware,date_published,link
0,"Stanley Electric U.S. (stanleyelectricus.com),...",Royal,2023-04-11,https://twitter.com/FalconFeedsio/status/16457...
1,Arrowhead Regional Computing Consortium (arcc....,LockBit,2023-04-11,https://twitter.com/FalconFeedsio/status/16457...
2,"Nature Path Foods (naturespath.com),a family-r...",Royal,2023-04-11,https://twitter.com/FalconFeedsio/status/16457...
3,"Grupo Euromotors (euromotors.com.pe),",LockBit,2023-04-11,https://twitter.com/FalconFeedsio/status/16457...
4,Romanian Association for International Road Tr...,LockBit,2023-04-11,https://twitter.com/FalconFeedsio/status/16457...
5,"Big Ass Fans (bigassfans.com),a company that d...",Royal,2023-04-11,https://twitter.com/FalconFeedsio/status/16457...
6,"Dealer Information Systems (disltd.ca),",LockBit,2023-04-11,https://twitter.com/FalconFeedsio/status/16457...
7,"GREGOIRE (gregoire.fr),",LockBit,2023-04-11,https://twitter.com/FalconFeedsio/status/16457...
8,Alvaria (alvaria.com),Royal,2023-04-11,https://twitter.com/FalconFeedsio/status/16457...
9,"Atlantic, Gulf & Pacific Company (agpglobal.com),",LockBit,2023-04-11,https://twitter.com/FalconFeedsio/status/16457...


In [7]:
df_combined = pd.concat([rw_df,rw_csv],ignore_index=True)
df_combined.drop_duplicates('link',inplace=True)
rw_df.to_csv('rw_csv.csv', mode='w', header=True, index=False)